<a href="https://colab.research.google.com/github/nitsundon/Load-Forecast/blob/main/ReadDSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install tesseract-ocr
!pip install requests pillow pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import drive.MyDrive.Libraries.SV as sv
import drive.MyDrive.Libraries.DatabseHandler as dbh

In [ ]:
sv.getDemand()

'2025.04.04 22:35 --> 26147'

In [4]:
dbh.DBHandler()